ML4VA

Helen Yao, Eva Butler, and Larissa Cybyk

In [1]:
import pandas as pd
import numpy as np

C:\Users\l3n3c\AppData\Local\Temp\ipykernel_50240\2162656668.py:1: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


Data Cleaning & Visualization

In [73]:
# load data

traffic_volume = pd.read_csv('./VDOT_Traffic_Volume.csv')

In [74]:
traffic_volume.head()

traffic_volume.columns
print(traffic_volume.shape)
nulls = traffic_volume.isnull().sum()
print(nulls)

(121679, 42)
OBJECTID                             0
LINKID                               0
DATA_DATE                            0
ROUTE_COMMON_NAME                    0
ROUTE_NAME                           0
HTRIS_ID                           745
ROUTE_ALIAS                          0
START_LABEL                          0
END_LABEL                            0
ADT                                  3
ADT_QUALITY                          0
PERCENT_4_TIRE                   94712
PERCENT_BUS                      94712
PERCENT_TRUCK_2_AXLE             94712
PERCENT_TRUCK_3_AXLE             94712
PERCENT_TRUCK_1_TRAIL            94712
PERCENT_TRUCK_2_TRAIL            94712
CLASS_QUALITY_CODE                   0
K_FACTOR                         92351
K_FACTOR_QUALITY_CODE               63
DIRECTION_FACTOR                 92371
AAWDT                            91950
AAWDT_QUALITY_CODE                   6
LOC_COMP_DIRECTIONALITY_NAME       196
ROUTE_FROM_MEASURE                   0
FROM_JURISDI

In [75]:
# dropping null columns / columns with a lot of missing data

to_drop = []
for col in traffic_volume.columns:
    if nulls[col] > 90000:
        to_drop.append(col)
print(to_drop)
traffic_volume = traffic_volume.drop(columns=to_drop)

['PERCENT_4_TIRE', 'PERCENT_BUS', 'PERCENT_TRUCK_2_AXLE', 'PERCENT_TRUCK_3_AXLE', 'PERCENT_TRUCK_1_TRAIL', 'PERCENT_TRUCK_2_TRAIL', 'K_FACTOR', 'DIRECTION_FACTOR', 'AAWDT', 'PEAK_SINGLE_UNIT_TRUCK', 'PEAK_COMBINATION_TRUCK', 'DATA_CUSTODIAN_EMAIL', 'METADATA_URL']


In [76]:
traffic_volume.isnull().sum()

OBJECTID                          0
LINKID                            0
DATA_DATE                         0
ROUTE_COMMON_NAME                 0
ROUTE_NAME                        0
HTRIS_ID                        745
ROUTE_ALIAS                       0
START_LABEL                       0
END_LABEL                         0
ADT                               3
ADT_QUALITY                       0
CLASS_QUALITY_CODE                0
K_FACTOR_QUALITY_CODE            63
AAWDT_QUALITY_CODE                6
LOC_COMP_DIRECTIONALITY_NAME    196
ROUTE_FROM_MEASURE                0
FROM_JURISDICTION                19
ROUTE_TO_MEASURE                  0
TO_JURISDICTION                  13
EVENT_SOURCE_ID                   0
EVENT_LOCATION_ID                 0
EVENT_COMPONENT_ID                0
MAXIMUMSCALEDENOMINATOR           9
ROUTE_COMMON_NAME_ADT             0
FROM_DISTRICT                   431
TO_DISTRICT                     544
RTE_TYPE_CD                       0
RTE_ID                      

In [77]:
# adding day of week feature
import re
import datetime
import calendar

def convert_date(date):
    stripped = date.split('/')
    try:
        given_date = datetime.datetime(int(stripped[2][:4]), int(stripped[0]), int(stripped[1]))
        
    except:
        given_date = datetime.datetime(int(stripped[0]), int(stripped[1]), int(stripped[2][:2]))
    return given_date
        
    
def day_of_week(given_date):
    day_num = given_date.weekday()
    day = 'sunday'
    if day_num == 0:
        day = 'monday'
    elif day_num == 1:
        day = 'tuesday'
    elif day_num == 2:
        day = 'wednesday'
    elif day_num == 3:
        day = 'thursday'
    elif day_num == 4:
        day = 'friday'
    elif day_num == 5:
        day = 'saturday'
    
    return day

def get_day(given_date):
    return given_date.day

def get_month(given_date):
    return given_date.month

def get_year(given_date):
    return given_date.year

** going to drop data_date once datasets are merged

In [78]:
traffic_volume['DATA_DATE'] = traffic_volume['DATA_DATE'].apply(convert_date) #converting data date into datetime

In [79]:
traffic_volume['DAY_WEEK'] = traffic_volume['DATA_DATE'].apply(day_of_week)
traffic_volume['DAY'] = traffic_volume['DATA_DATE'].apply(get_day)
traffic_volume['MONTH'] = traffic_volume['DATA_DATE'].apply(get_month)
traffic_volume['YEAR'] = traffic_volume['DATA_DATE'].apply(get_year)

In [80]:
traffic_volume.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 121679 entries, 0 to 121678
Data columns (total 33 columns):
 #   Column                        Non-Null Count   Dtype         
---  ------                        --------------   -----         
 0   OBJECTID                      121679 non-null  int64         
 1   LINKID                        121679 non-null  int64         
 2   DATA_DATE                     121679 non-null  datetime64[ns]
 3   ROUTE_COMMON_NAME             121679 non-null  object        
 4   ROUTE_NAME                    121679 non-null  object        
 5   HTRIS_ID                      120934 non-null  object        
 6   ROUTE_ALIAS                   121679 non-null  object        
 7   START_LABEL                   121679 non-null  object        
 8   END_LABEL                     121679 non-null  object        
 9   ADT                           121676 non-null  float64       
 10  ADT_QUALITY                   121679 non-null  object        
 11  CLASS_QUALITY

merge datasets BEFORE dropping

In [ ]:
# merging
new_df = pd.merge(
    left=traffic_volume, 
    right=crashes,
    how='left',
    left_on=['DATA_DATE', 'ROUTE_NAME'],
    right_on=['DATA_DATE', 'ROUTE_NAME']
)

dropping route identifiers, addresses, etc

In [62]:
to_drop = ['ROUTE_COMMON_NAME', 'ROUTE_NAME', 'ROUTE_ALIAS', 'END_LABEL', 'START_LABEL', 'ADT_QUALITY', 'CLASS_QUALITY_CODE', 'K_FACTOR_QUALITY_CODE', 'AAWDT_QUALITY_CODE', 'MAXIMUMSCALEDENOMINATOR', 'ROUTE_COMMON_NAME_ADT']

In [63]:
traffic_volume = traffic_volume.drop(columns=to_drop)

In [66]:
traffic_volume = traffic_volume.drop(columns=['HTRIS_ID'])

In [67]:
traffic_volume.head()

,OBJECTID,LINKID,DATA_DATE,ADT,LOC_COMP_DIRECTIONALITY_NAME,ROUTE_FROM_MEASURE,FROM_JURISDICTION,ROUTE_TO_MEASURE,TO_JURISDICTION,EVENT_SOURCE_ID,...,EVENT_COMPONENT_ID,FROM_DISTRICT,TO_DISTRICT,RTE_TYPE_CD,RTE_ID,Shape__Length,DAY,DAY_WEEK,MONTH,YEAR
0,1,795997,2011-06-07,490.0,Bidirectional,0.000,Frederick County,0.24,Frederick County,795997,...,1,Staunton,Staunton,SC,2532756,478.381712,7,tuesday,6,2011
1,2,709217,2019-12-10,10.0,Bidirectional,0.620,Campbell County,0.64,Campbell County,709217,...,1,Lynchburg,Lynchburg,SC,1733670,47.494224,10,tuesday,12,2019
2,3,720637,2021-11-29,150.0,Bidirectional,3.180,Dickenson County,7.55,Dickenson County,720637,...,1,Bristol,Bristol,SC,1857885,8693.982351,29,monday,11,2021
3,4,755648,2020-07-30,47.0,Bidirectional,2.000,Nelson County,5.40,Nelson County,755648,...,1,Lynchburg,Lynchburg,SC,2111709,6516.476138,30,thursday,7,2020
4,5,776797,2017-07-07,30.0,Bidirectional,2.031,Washington County,2.08,Washington County,776797,...,1,Bristol,Bristol,SC,1594318,100.504177,7,friday,7,2017


In [68]:
traffic_volume.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 121679 entries, 0 to 121678
Data columns (total 21 columns):
 #   Column                        Non-Null Count   Dtype         
---  ------                        --------------   -----         
 0   OBJECTID                      121679 non-null  int64         
 1   LINKID                        121679 non-null  int64         
 2   DATA_DATE                     121679 non-null  datetime64[ns]
 3   ADT                           121676 non-null  float64       
 4   LOC_COMP_DIRECTIONALITY_NAME  121483 non-null  object        
 5   ROUTE_FROM_MEASURE            121679 non-null  float64       
 6   FROM_JURISDICTION             121660 non-null  object        
 7   ROUTE_TO_MEASURE              121679 non-null  float64       
 8   TO_JURISDICTION               121666 non-null  object        
 9   EVENT_SOURCE_ID               121679 non-null  int64         
 10  EVENT_LOCATION_ID             121679 non-null  int64         
 11  EVENT_COMPONE

In [71]:
for feature in traffic_volume.select_dtypes("object"):
    print(feature, len(traffic_volume[feature].unique()))

LOC_COMP_DIRECTIONALITY_NAME 4
FROM_JURISDICTION 185
TO_JURISDICTION 185
FROM_DISTRICT 10
TO_DISTRICT 10
RTE_TYPE_CD 8
DAY_WEEK 7
